In [1]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from scipy.stats import poisson,skellam
import math
import os

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Upload csv season files and cleaup data
century=2000
start_year = 2
end_year = 23

df = {}
data = {}

Name =[]

os.chdir("../")

for i in range(start_year, end_year):
    year1 = century+i
    year2 = century+i+1
    
    txt = str(year1)+"-"+str(year2)
    Name.append(txt)
    
    file_name = 'EPL_'+str(year1)+"_"+str(year2)+".csv"
    
    txt_path = os.getcwd()+"\\Data\\"+file_name
    
    data[txt] = i
    
    df[txt] = pd.read_csv(txt_path, index_col="Date", encoding= 'unicode_escape', parse_dates=True, infer_datetime_format=True, on_bad_lines='skip')

    df[txt] = df[txt][df[txt].index.notnull()]
    
    df[txt]['Season'] = txt
    
    df[txt]['Year'] = df[txt].index.year
    df[txt]['Month'] = df[txt].index.month
    df[txt]['Day'] = df[txt].index.day
    
    year = df[txt]['Year'].to_list()
    mnth = df[txt]['Month'].to_list()
    day = df[txt]['Day'].to_list()
    
    df[txt].drop(['Year', 'Month', 'Day'], inplace=True, axis=1)
    
    
    adj_month = []
    adj_day = []
    
    count = len(mnth)
    
    for j in range(0, count):
        if((day[j] in range(1,13)) and (mnth[j] in range(0,13))):
            adj_month.append(int(day[j]))
        else:
            adj_month.append(int(mnth[j]))
    
    for k in range(0, count):
        if((day[k] in range(1,13)) and (mnth[k] in range(0,13))):
            adj_day.append(int(mnth[k]))
        else:
            adj_day.append(int(day[k]))
    
   
    year1 = []
    for l in year:
        year1.append(int(l))
    
    date = []
    
    for i in range(0, count):
        d1 = dt.datetime(year[i], adj_month[i], adj_day[i])
        date.append(d1)
    
    df[txt].index = date
    
    df[txt].index.names = ['Date']
    
    #df[txt] = df[txt].sort_index()
    
    df[txt] = df[txt][['Season', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HS', 'AS', 'HST','AST','B365H', 'B365D', 'B365A']]  # We'll only look at BET365 Odds for now
    
    
path_update = os.getcwd()+"\\Processed_Data"  
os.chdir(path_update)

In [3]:
#Convert the data to a dataframe that can be used for furtehr analysis 
s1 = '2002-2003' 
s2 = '2022-2023' 

i1 = data[s1]-2
i2 = data[s2]-1

try:
    del seasons
except:
    pass

    
seasons = df[s1]

if(s1==s2):
    pass
else:
    for i in range(i1+1, i2):
        seasons = seasons.append(df[Name[i]])

In [4]:
#Function to categorise the game result
def res(x):
    if (x == 'H'):
        return 1
    else:
        return 0

In [5]:
#Calculate the true probabilities and fair odds from bookmakers posted prices and also categorize the game result
seasons['Result'] = seasons.apply(lambda x: res(x['FTR']), axis=1)

seasons['%_Book_Home_Win'] = seasons.apply(lambda x: round((1/x['B365H']),3), axis=1)
seasons['%_Book_Draw'] = seasons.apply(lambda x: round((1/x['B365D']),3), axis=1)
seasons['%_Book_Away_Win'] = seasons.apply(lambda x: round((1/x['B365A']),3), axis=1)

seasons['Total_Probability'] = seasons['%_Book_Home_Win'] + seasons['%_Book_Draw'] + seasons['%_Book_Away_Win']

seasons['mktProb_homeWin'] = round(seasons['%_Book_Home_Win']/seasons['Total_Probability'],2)
seasons['mktProb_Draw'] = round(seasons['%_Book_Draw']/seasons['Total_Probability'],2)
seasons['mktProb_awayWin'] = round(seasons['%_Book_Away_Win']/seasons['Total_Probability'],2)
seasons['mktProb_NOThomeWin'] = round(seasons['mktProb_Draw'] + seasons['mktProb_awayWin'],2)

seasons['True_Win_Odds'] = round((1/seasons['mktProb_homeWin']),2)
seasons['True_Draw_Odds'] = round((1/seasons['mktProb_Draw']),2)
seasons['True_Away_Odds'] = round((1/seasons['mktProb_awayWin']),2)


seasons = seasons.drop(columns=['%_Book_Home_Win', '%_Book_Draw', '%_Book_Away_Win', 'Total_Probability', 
                                'mktProb_Draw', 'mktProb_awayWin', 'True_Win_Odds', 'True_Draw_Odds', 'True_Away_Odds'])

In [6]:
# Function to calculate the Probability
def Probability(rating1, rating2):
 
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 400))
 
# function to calculate Elo rating
# K is a constant.
# d determines whether
# Player A wins or Player B.
def EloRating(Ra, Rb, K, d):
  
 
    # To calculate the Winning
    # Probability of Player B
    Pb = Probability(Ra, Rb)
 
    # To calculate the Winning
    # Probability of Player A
    Pa = Probability(Rb, Ra)
 
    # Case -1 When Player A wins
    # Updating the Elo Ratings
    if (d == 1):
        Ra = Ra + K * (1 - Pa)
        Rb = Rb + K * (0 - Pb)
        
        
    # Case -2 When Player A wins
    # Updating the Elo Ratings     
    elif (d == 0) :
        Ra = Ra + K * (0.5 - Pa)
        Rb = Rb + K * (0.5 - Pb)
     
 
    # Case -3 When Player B wins
    # Updating the Elo Ratings
    else:
        Ra = Ra + K * (0 - Pa)
        Rb = Rb + K * (1 - Pb)
        
    return Ra, Rb

In [7]:
# Calculate elo Rating for each season and associated win probability for home and away team for each season
s0 = seasons['Season'].unique()
count1 = len(s0)

gt = seasons.groupby('Season')

home_elo = []
away_elo = []

home_prob = []
away_prob = []


for w in s0:
    v = gt.get_group(w)
    
    teams_uniq = v['HomeTeam'].unique()
        
    elo=[]
    teams=[]
    
    for l in teams_uniq:
        elo.append(1600)
        teams.append(l)
    
    homeTeam = v['HomeTeam'].unique()
    awayTeam = v['AwayTeam'].unique()
        
    for t in range(0, v.shape[0]):
        indexhome = teams.index(v.iat[t,1])
        indexaway = teams.index(v.iat[t,2])
    
        home_elo.append(elo[indexhome])
        away_elo.append(elo[indexaway])
        
        home_prob.append(Probability(elo[indexaway], elo[indexhome]))
        away_prob.append(Probability(elo[indexhome], elo[indexaway]))
    
        Ra = elo[indexhome]
        Rb = elo[indexaway]
        k = 24
    
        result = v.iat[t,5]
    
        if(result == 'H'):
            d = 1
        elif(result == 'D'):
            d = 0
        else:
            d = 2
        
        
        score = EloRating(Ra, Rb, k, d)
    
        elo[indexhome] = score[0]
        elo[indexaway] = score[1]
        
        
seasons['Home_Elo_Rating'] = home_elo
seasons['Away_Elo_Rating'] = away_elo
seasons['eloProb_homeWin'] = home_prob
seasons['eloProb_awayWin'] = away_prob

In [8]:
#Function to Calculate Points for Target Team 
def pts(h, r, team):
    if((h == team) & (r == 'H')):
        pts = 3
    elif((h != team) & (r == 'A')):
        pts = 3
    elif(r =='D'):
        pts = 1
    else:
        pts = 0
    
    return pts

In [9]:
## Simulate match using Poisson Distribution Model
def simulate_match(home_goals_avg, away_goals_avg, max_goals=10):
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [home_goals_avg, away_goals_avg]]
    return(np.outer(np.array(team_pred[0]), np.array(team_pred[1])))


In [10]:
def matchProb(homePower, awayPower):
    matrix = simulate_match(homePower,awayPower)
    probHomeWin = np.sum(np.tril(matrix, -1))
    probDraw = np.sum(np.diag(matrix))
    probAwayWin = np.sum(np.triu(matrix, 1))
    
    results = [probHomeWin, probDraw, probAwayWin]
    return results

In [11]:
#Here we loop through each unique team for the 20 seasons considered and calculated team statistics
t1 = seasons['HomeTeam'].unique()
team_df = pd.DataFrame()
#gameLag = 6

for y in t1:

    team = y
    teami = seasons[(seasons['HomeTeam']==team) | (seasons['AwayTeam']==team)]
    teami['targetTeam'] = team
    teami['targertTeamHome'] = teami.apply(lambda x: 1 if (x['HomeTeam'] == team) else 0, axis=1)
    teami['targetPoints'] = teami.apply(lambda x: pts(x['HomeTeam'], x['FTR'], team), axis=1)



    s1 = teami['Season'].unique()
    
    count1 = len(s1)
    gs = teami.groupby('Season')

    for i in s1:
        k = gs.get_group(i)
    
    
        k['teamGoals'] = k.apply(lambda x: x['FTHG'] if x['HomeTeam'] == team else x['FTAG'], axis=1)

        k['teamShots'] = k.apply(lambda x: x['HS'] if x['HomeTeam'] == team else x['AS'], axis=1)
        
        k['teamTShots'] = k.apply(lambda x: x['HST'] if x['HomeTeam'] == team else x['AST'], axis=1)
        
        
        k['awayGoals'] = k.apply(lambda x: x['FTAG'] if x['HomeTeam'] == team else x['FTHG'], axis=1)

        k['awayShots'] = k.apply(lambda x: x['AS'] if x['HomeTeam'] == team else x['HS'], axis=1)
        
        k['awayTShots'] = k.apply(lambda x: x['AST'] if x['HomeTeam'] == team else x['HST'], axis=1)
        
   
        team_df = team_df.append(k)
        
team_merge = team_df[['Season','targetTeam']]

In [12]:
#Merge the seasons dataframe withe the team_merged dataframe (the 2'nd df has the additional team power statistics)
team_merge['gameDate'] = team_merge.index
seasons['gameDate'] = seasons.index

#Rename columns for home team and away team
merged_data = pd.merge(seasons, team_merge, how='left', left_on=['Season', 'gameDate', 'HomeTeam'], right_on=['Season', 'gameDate', 'targetTeam'])
merged_stats = pd.merge(merged_data, team_merge, how='left', left_on=['Season', 'gameDate', 'AwayTeam'], right_on=['Season', 'gameDate', 'targetTeam'])

merged_stats = merged_stats.drop(columns=['targetTeam_x', 'targetTeam_y'])

merged_stats.index = merged_stats['gameDate']
merged_stats.index.names = ['Date']

merged_stats.to_csv('merged_stats.csv')
seasons.to_csv('seasons.csv')
team_merge.to_csv('team_merge.csv')